In [1]:
from mesa import Agent, Model
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid
from mesa.batchrunner import BatchRunner
from sklearn.neighbors import KernelDensity

import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import random

# Age Distribution

Mortality rates are taken from https://www.abs.gov.au/statistics/people/population/life-tables/latest-release
Table 1: Life tables, states, territories and Australia - 2017-2019

initial age distribution is taken from https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/3101.0Jun%202019?OpenDocument  Population by Age and Sex Tables 

In [4]:
def generate_age_density():
    #agedist has two columns: "AgeGroup" and "Prop"
    #uses kernel density
    kernelpopsize=3000
    agedist=pd.read_csv('Data/AgeDistribution.csv')
    agelist=list(range(2,87,5))+[90]
    proplist=list(agedist["Prop"].to_numpy())
    num_in_groups= [element * kernelpopsize for element in proplist]
    df_freq=np.repeat(agelist, num_in_groups, axis=0)
    df_freq=[[element] for element in df_freq]

    # Do kernel density estimation
    #x = np.linspace(0, 100, 1000)[:, np.newaxis]

    kd = KernelDensity(kernel='gaussian', bandwidth=5).fit(df_freq)
    return kd


kd=generate_age_density()
#assign death rates
deathrates=pd.read_csv('Data/deathrates.csv')
#timediff=1
#display([x for x in deathrates['death_rate'][:-1]])
deathrates=np.array([x for x in deathrates['death_rate'][:-1]])
#display(deathrates[101])

In [ ]:
def sample_ages(popsize,kd, plot=False):
    while True:
        #sample from density
        generated_ages=kd.sample(round(popsize*1.5)) #,random_state=0)
        generated_ages=[round(x[0]) for x in generated_ages]
        myages=[x for x in generated_ages if (x > 0 and x<=100)  ]
        if len(myages)>=popsize:
            break
    myages=random.sample(myages, popsize)   
    if(plot):
        plt.hist(myages,alpha=0.5)
    return myages